### Import

In [59]:
import gzip
import json
import random
import time
from nltk import word_tokenize

# Imports for the synonym dictionary 
import requests
from bs4 import BeautifulSoup


### Read in the data

In [3]:
#here
def reading_in(path):
    thing=[]
    for line in gzip.open(path):
        review_data = json.loads(line)
        subthing = dict()
        for key in review_data:
            subthing[key]= review_data[key]
        thing.append(subthing)
    return thing

In [4]:
train = reading_in('../data/classification/music_reviews_train.json.gz')
#dev = reading_in('../data/classification/music_reviews_dev.json.gz') # do not use!
#test = reading_in('../data/classification/music_reviews_test_masked.json.gz') #do not use!

In [5]:
train_x = [i['reviewText'] for i in train if "reviewText" in i.keys() and "sentiment" in i.keys()]
train_y =  [i['sentiment'] for i in train if "reviewText" in i.keys() and "sentiment" in i.keys()]

### Preparing the data

In [44]:
train_x_split = [sen.split(' ') for sen in train_x]

"""
Here we want a loop for replacing the words
"""

train_x_back = [' '.join(sen) for sen in train_x_split]

# Webscrape synonyms

In [ ]:
##Alternative to PyDictionary
##THIS IS COPIED FROM SLACK
# https://stackoverflow.com/questions/52910297/pydictionary-word-has-no-synonyms-in-the-api

def synonyms(term):
    response = requests.get('https://www.thesaurus.com/browse/{}'.format(term))
    soup = BeautifulSoup(response.text, 'lxml')
    soup.find('section', {'class': 'css-17ofzyv e1ccqdb60'})
    return [span.text for span in soup.findAll('a', {'class': 'css-1kg1yv8 eh475bn0'})] # 'css-1gyuw4i eh475bn0' for less relevant synonyms

word = "I"
print(synonyms(word))

['he ', 'it ', 'she ', 'sie ', 'they ', 've ', 'xe ', 'you ', 'ze ', 'zie ']


In [ ]:
def gen_synth(sentence,prob = 0.5):
    new_sen = []
    for word in sentence:
        threshold = random.random()
        if prob > threshold:
            #print(threshold) can remove later kept for debugging
            syn = synonyms(word)
            if len(syn) != 0:
                #print("selected", word) can remove later kept for debugging
                new_sen.append(random.choice(syn))
            else:new_sen.append(word)
        else:
            new_sen.append(word)
    return new_sen

In [ ]:
len(train_x)

99946

In [ ]:
from tqdm import tqdm

In [ ]:
print([0,1,2,3,4,5][2:4])#indexing sanity check can delecte later

[2, 3]


In [ ]:
#for i in tqdm(train_x_split[:1000]):#running with 0:1000 dont uncomment as will mess up file 
#    with open ("../data/synthetic1.txt","a") as f:
#        f.write(' '.join(gen_synth(i))+"\n")
#        time.sleep(5)

 28%|██▊       | 275/1000 [36:04<1:29:34,  7.41s/it]

100%|██████████| 1000/1000 [2:44:06<00:00,  9.85s/it]


In [ ]:
print(train_x_split[6])

['I', 'love', 'all', 'of', 'his', 'music!!']


In [ ]:
print(gen_synth(train_x_split[6]))

['you ', 'embrace ', 'only ', 'of', 'his', 'music!!']


# Wordnet

In [61]:
#importing NLTK corpus synonym dictionary and checking the outputs
import nltk
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk.corpus import wordnet
from nltk import pos_tag
syns = wordnet.synsets("furniture")
syns

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[Synset('furniture.n.01')]

In [62]:
def change(sentence):
    sentence = word_tokenize(sentence)    #tokenise
    sentence = nltk.pos_tag(sentence)    #pos sentence here
    print(sentence)
    for word in sentence:
        pass# if relevant pos then replace with a syn
    #formatting
    return sentence

In [63]:
change(train_x[0])

[('So', 'RB'), ('creative', 'JJ'), ('!', '.'), ('Love', 'VB'), ('his', 'PRP$'), ('music', 'NN'), ('-', ':'), ('the', 'DT'), ('words', 'NNS'), (',', ','), ('the', 'DT'), ('message', 'NN'), ('!', '.'), ('Some', 'DT'), ('of', 'IN'), ('my', 'PRP$'), ('favorite', 'JJ'), ('songs', 'NNS'), ('on', 'IN'), ('this', 'DT'), ('CD', 'NN'), ('.', '.'), ('I', 'PRP'), ('should', 'MD'), ('have', 'VB'), ('bought', 'VBN'), ('it', 'PRP'), ('years', 'NNS'), ('ago', 'RB'), ('!', '.')]


[('So', 'RB'),
 ('creative', 'JJ'),
 ('!', '.'),
 ('Love', 'VB'),
 ('his', 'PRP$'),
 ('music', 'NN'),
 ('-', ':'),
 ('the', 'DT'),
 ('words', 'NNS'),
 (',', ','),
 ('the', 'DT'),
 ('message', 'NN'),
 ('!', '.'),
 ('Some', 'DT'),
 ('of', 'IN'),
 ('my', 'PRP$'),
 ('favorite', 'JJ'),
 ('songs', 'NNS'),
 ('on', 'IN'),
 ('this', 'DT'),
 ('CD', 'NN'),
 ('.', '.'),
 ('I', 'PRP'),
 ('should', 'MD'),
 ('have', 'VB'),
 ('bought', 'VBN'),
 ('it', 'PRP'),
 ('years', 'NNS'),
 ('ago', 'RB'),
 ('!', '.')]

In [48]:
wn_x_train = [change(i) for i in train_x_split]


True

In [ ]:
v = ['I', 'love', 'all', 'of', 'his', 'music!!']
for word in v:
    print(wordnet.synsets(word))

[Synset('iodine.n.01'), Synset('one.n.01'), Synset('i.n.03'), Synset('one.s.01')]
[Synset('love.n.01'), Synset('love.n.02'), Synset('beloved.n.01'), Synset('love.n.04'), Synset('love.n.05'), Synset('sexual_love.n.02'), Synset('love.v.01'), Synset('love.v.02'), Synset('love.v.03'), Synset('sleep_together.v.01')]
[Synset('all.a.01'), Synset('all.s.02'), Synset('wholly.r.01')]
[]
[]
[]


[]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b88dfe01-c7e1-473c-bcfd-798313fc6522' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>